In [ ]:
%load_ext autoreload
%autoreload

from src.utils import *
from src.kgrams import *
from src.levenshtein import *
from sklearn import svm
import time
from tqdm import tqdm_notebook
import os

## Baseline using Spectrum kernel

In [ ]:
%time cum_spectrum_31_kernels = precomputed_kernels(cum_spectrum, 'cum_spectrum_31', k=31)

In [ ]:
%time svm_kernels(cum_spectrum_31_kernels, 'cum_spectrum_31')

The computations are slowed down over $k=13$.

# Levenshtein Kernel

### Kernel Computing

In [ ]:
%time levenshtein_distances = precomputed_kernels(levenshtein_distance, 'levenshtein_distance', numeric=False)

In [ ]:
def distance_to_kernel(i, d, α):
    return 1 / (1 + d ** α)

levenshtein_kernels = transform_kernels([levenshtein_distances], distance_to_kernel, α=.25)

In [ ]:
svm_kernels(levenshtein_kernels, 'levenshtein')

In [ ]:
αs = [.7, .7, .7]
svm_kernels(transform_kernels(
    [levenshtein_kernels, cum_spectrum_31_kernels], 
    lambda i, x, y: (1-αs[i]) * x + αs[i] * y), 'mixed_sum')

In [ ]:
svm_kernels(transform_kernels(
    [levenshtein_kernels, cum_spectrum_31_kernels], 
    lambda _, x, y: (1 + .5 * x) * y), 'mixed_product')